In [116]:
import pandas as pd
import numpy as np

In [117]:
df = pd.read_csv('../database/alunos-ativos-da-pos-graduacao.csv', sep=';', usecols=['nome_aluno']).sample(frac=1).reset_index(drop=True)
df = df.rename(columns={'nome_aluno' : 'nome'})

pd.options.display.max_rows = 6

In [118]:
#Atribuindo sexo considerando nomes com maior frequencia
df_nomes_frequencia = df['nome'].map(lambda y: y.split()[0]).value_counts()

nomes_masculinos = ['Lucas', 'João', 'Gabriel', 'Matheus', 'Rafael', 'Guilherme', 'Gustavo', 'Bruno', 'Luiz', 'Paulo', 'Pedro', 'André', 'José', 'Rodrigo', 'Thiago', 'Felipe', 'Daniel', 'Marcelo', 'Eduardo']
nomes_femininos = ['Ana', 'Maria', 'Mariana', 'Bruna', 'Gabriela', 'Camila', 'Fernanda', 'Amanda', 'Aline', 'Leticia', 'Larissa', 'Juliana', 'Jéssica', 'Beatriz', 'Marina']

df['sexo'] = None

df.loc[df['nome'].str.split().str[0].isin(nomes_masculinos), 'sexo'] = 'M'
df.loc[df['nome'].str.split().str[0].isin(nomes_femininos), 'sexo'] = 'F'

lista_sexo_aleatorio = np.random.choice(['M', 'F'], size=df['sexo'].isnull().sum())
df.loc[df['sexo'].isnull(), 'sexo'] = lista_sexo_aleatorio

In [119]:
#Define data de nascimento e idade
df['nascimento'] = None
df.loc[df['nascimento'].isnull(), 'nascimento'] = np.random.choice(pd.date_range(start='1910-01-01', end='2019-12-31').strftime('%Y-%m-%d'), size=len(df))

df['idade'] = (pd.Timestamp.now() - pd.to_datetime(df['nascimento'])).dt.days // 365

In [120]:
#define altura com influencia da idade e sexo
df['altura'] = np.random.randint(120, 210, size=len(df))
df.loc[df['idade'] <= 18, 'altura'] -= (25 * (1 - (df['idade'] / 18) ** 2 )).round() #quanto menor a idade, maior o decrescimo
df.loc[df['idade'] >= 60, 'altura'] -= (30 * ((df['idade'] - 60) / (df['idade'].max() - 60)) ** 2).round() #quanto maior a idade, maior o decrescimo

df.loc[df['sexo'] == 'F', 'altura'] -= np.random.randint(1, 8)
df.loc[df['sexo'] == 'M', 'altura'] += np.random.randint(1, 8)

In [121]:
#define o peso com influencia da idade e sexo
df['peso'] = np.round(np.random.uniform(25.0, 150, size=len(df)), decimals=2)
df.loc[df['idade'] <= 18, 'peso'] -= (8 * (1 - (df['idade'] / 18) ** 2 )).round(decimals=2)
df.loc[df['idade'] >= 60, 'peso'] -= (15 * ((df['idade'] - 60) / (df['idade'].max() - 60)) ** 2).round(decimals=2)

df.loc[df['sexo'] == 'F', 'peso'] -= (df['peso'] * 0.05).round(decimals=2)
df.loc[df['sexo'] == 'M', 'peso'] += (df['peso'] * 0.05).round(decimals=2)

In [122]:
# define pressao arterial
df['pressao-arterial-sistolica'] = 10 * np.random.randint(5, 23, size=len(df))
df['pressao-arterial-diastolica'] = 10 * np.random.randint(3, (df['pressao-arterial-sistolica']/10), size=len(df))

df.loc[df['idade'] <= 18, ['pressao-arterial-sistolica', 'pressao-arterial-diastolica']] -= (10 * np.random.randint(1, 4))

In [123]:
# define glicemia com aumento na terceira idade
df['glicemia'] = 5 * np.random.randint(8, 44, size=len(df))
df.loc[df['idade'] >= 60, 'glicemia'] += 5 * np.random.randint(1, 4)
df

,nome,sexo,nascimento,idade,altura,peso,pressao-arterial-sistolica,pressao-arterial-diastolica,glicemia
0,Luana de Moura Coelho,M,1992-08-26,31,183,123.99,100,40,125
1,Rafael Tofoli Sereicikas,M,1982-12-14,41,190,104.96,60,40,85
2,Elaine Pinto Sousa,M,1965-05-21,59,202,144.97,100,60,55
...,...,...,...,...,...,...,...,...,...
3882,Pedro Henrique Bergamo Bertolli.,M,1920-08-08,103,123,78.67,180,150,125
3883,Vanuza dos Santos Lima,M,2004-04-27,20,147,78.42,110,50,125
3884,José Carlos dos Santos,M,1964-01-31,60,176,72.85,220,140,95


In [124]:
# Gera um valor de frequencia cardiaca com influencia da idade
df['frequencia-cardiaca'] = np.random.randint(30, 210, size=len(df))

df.loc[df['idade'] <= 18, 'frequencia-cardiaca'] += (np.random.randint(20, 40) * (1 - (df['idade'] / 18) ** 2 )).round()
df.loc[df['idade'] >= 60, 'frequencia-cardiaca'] += (np.random.randint(10, 30) * ((df['idade'] - 60) / (df['idade'].max() - 60)) ** 2).round()

df

,nome,sexo,nascimento,idade,altura,peso,pressao-arterial-sistolica,pressao-arterial-diastolica,glicemia,frequencia-cardiaca
0,Luana de Moura Coelho,M,1992-08-26,31,183,123.99,100,40,125,120
1,Rafael Tofoli Sereicikas,M,1982-12-14,41,190,104.96,60,40,85,161
2,Elaine Pinto Sousa,M,1965-05-21,59,202,144.97,100,60,55,178
...,...,...,...,...,...,...,...,...,...,...
3882,Pedro Henrique Bergamo Bertolli.,M,1920-08-08,103,123,78.67,180,150,125,58
3883,Vanuza dos Santos Lima,M,2004-04-27,20,147,78.42,110,50,125,55
3884,José Carlos dos Santos,M,1964-01-31,60,176,72.85,220,140,95,157


In [125]:
"""
df['circunferencia-cintura']
df['diabetes']
df['doenca-cronica']
df['tabagismo']
df['colesterol']
df['funcao-renal']
df['alergia']
df['duracao-sono']"""

"\ndf['circunferencia-cintura']\ndf['diabetes']\ndf['doenca-cronica']\ndf['tabagismo']\ndf['colesterol']\ndf['funcao-renal']\ndf['alergia']\ndf['duracao-sono']"